## Processing the dataset

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#from Bio import SeqIO

pd.set_option('display.max_columns', 100)
plt.style.use('ggplot')

In [5]:
# Load the Excel file
xls = pd.ExcelFile('NIHMS1014772-supplement-Supplemental_2.xlsx')

# Get the sheet names
sheet_names = xls.sheet_names

print(f'Number of datasets in the file: ', len(sheet_names))

Number of datasets in the file:  44


In [6]:
sheet_names[0]

'BRCA1_HUMAN_BRCT'

In [7]:
df = pd.read_excel('NIHMS1014772-supplement-Supplemental_2.xlsx', sheet_name=sheet_names[0])
df

,Unnamed: 0,mutant,function_score,mutation_effect_prediction_vae_ensemble,mutation_effect_prediction_vae_1,mutation_effect_prediction_vae_2,mutation_effect_prediction_vae_3,mutation_effect_prediction_vae_4,mutation_effect_prediction_vae_5,mutation_effect_prediction_pairwise,mutation_effect_prediction_independent
0,0,S1631G,0.305708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,S1631C,0.133622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,S1631N,-0.610019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,S1631T,-0.895748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,S1631I,-1.180832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1617,1617,I1855T,-0.291519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1618,1618,I1855R,-0.464328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1619,1619,I1855K,-0.863844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1620,1620,I1855M,0.021941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## MIC - blat ecolx Tenaillon 2013
this dataset they evaluate the MIC required of amoxicillin to inhibit the growth of bacteria with the mutants of beta-lactamase TEM-1 - 1ZG4

In [32]:
#wt_seq = 'MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRIDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPVAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW'
wt_seq = str(SeqIO.read('BLAT_ECOLX_Tenaillon2013/rcsb_pdb_1ZG4.fasta', 'fasta').seq)
wt_seq[19:24-1]

'PVFA'

In [ ]:
data = pd.read_excel('/Users/luiz/Downloads/multi_layer_analysis/Data_process/BLAT_ECOLX_Tenaillon2013/sd01.xlsx')
#data['Mutation'].is_unique
data

In [ ]:
def mutated_seqs(wt, Residue, wt_AA, Mutant_AA):
    coor = int(Residue)-1
    sub = Mutant_AA
    wt_AA = wt_AA

    if wt[coor] == wt_AA:
        seq = wt[:coor] + sub + wt[coor+1:]

    return seq


mutated_seqs(B-lac_1ZG4, 1, 'M', 'X')

In [ ]:
# Define the function

def mutated_seqs(col):
    coor = int(col['Residue']) - 1
    sub = col['Mutant_AA']
    wt_AA = col['Wt_AA']

    if wt_seq[coor] == wt_AA:
        seq = wt_seq[:coor] + sub + wt_seq[coor + 1:]
        return seq
    else:
        return wt_seq

# Apply the function to each col in the DataFrame
data['sequences'] = data.apply(mutated_seqs, axis=1)

data

In [ ]:
data['ID'] = 'B-lac_1ZG4_' + data['Mutation'].astype(str)


cols = ['ID', 'sequences', 'Residue', 'Wt_AA', 'Mutant_nt', 'Mutant_AA', 'Mutation',
       'Mutation_Ambler', 'MIC_Score', '∆∆G_foldX', '∆∆G_PopMusic',
       'Accessibility', 'BLOSUM62_penalty', 'Active_site', 'X', 'Y', 'Z',
       'Maximum_Growth_Rate_AMX6', 'Maximum_Growth_Rate_AMX100', 'logVo',
       'logVo_OD610']

data = data[cols]
data

In [ ]:
#data.to_csv('B-lac_1ZG4_mutants_metadata.csv')

In [ ]:
# Create the fasta file
dict_seq = dict(zip(data['ID'], data['sequences']))

for k,v in dict_seq.items():
    with open('B-lac_1ZG4_mutants_data.fasta', 'a') as f:
        f.write(f'>{k}\n{v}\n') 

# Gal4 Selectivy score 64hs
Kitzman, J.O., etal. Massively parallel single-amino-acid mutagenesis. Nature Methods, 2015

Per-allele enrichment scores for Gal4 functional selection. Spreadsheet containing enrichment log2-ratio values for each mutant allele and round of selection.

In [3]:
seq_record = SeqIO.read("GAL4_SEL_C_64h/GAL4_sequence.fasta", "fasta")

gal4_wt = str(seq_record.seq)
gal4_wt

'MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKRSPLTRAHLTEVESRLERLEQLFLLIFPREDLDMILKMDSLQDIKALLTGLFVQDNVNKDAVTDRLASVETDMPLTLRQHRISATSSSEESSNKGQRQLTVSIDSAAHHDNSTIPLDFMPRDALHGFDWSEEDDMSDGLPFLKTDPNNNGFFGDGSLLCILRSIGFKPENYTNSNVNRLPTMITDRYTLASRSTTSRLLQSYLNNFHPYCPIVHSPTLMMLYNNQIEIASKDQWQILFNCILAIGAWCIEGESTDIDVFYYQNAKSHLTSKVFESGSIILVTALHLLSRYTQWRQKTNTSYNFHSFSIRMAISLGLNRDLPSSFSDSSILEQRRRIWWSVYSWEIQLSLLYGRSIQLSQNTISFPSSVDDVQRTTTGPTIYHGIIETARLLQVFTKIYELDKTVTAEKSPICAKKCLMICNEIEEVSRQAPKFLQMDISTTALTNLLKEHPWLSFTRFELKWKQLSLIIYVLRDFFTNFTQKKSQLEQDQNDHQSYEVKRCSIMLSDAAQRTVMSVSSYMDNHNVTPYFAWNCSYYLFNAVLVPIKTLLSNSKSNAENNETAQLLQQINTVLMLLKKLATFKIQTCEKYIQVLEEVCAPFLLSQCAIPLPHISYNNSNGSAIKNIVGSATIAQYPTLPEENVNNISVKYVSPGSVGPSPVPLKSGASFSDLVKLLSNRPPSRNSPVTIPRSTPSHRSVTPFLGQQQQLQSLVPLTPSALFGGANFNQSGNIADSSLSFTFTNSSNGPNLITTQTNSQALSQPIASSNVHDNFMNNEITASKIDDGNNSKPLSPGWTDQTAYNAFGITTGMFNTTTMDDVYNYLFDDEDTPPNPKKE'

In [4]:
data = pd.read_excel('GAL4_SEL_C_64h/41592_2015_BFnmeth3223_MOESM306_ESM.xlsx', sheet_name='SEL_C_64h', header=1)
data

/Users/luiz/anaconda3/envs/ML/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Residue,*,delInFrame,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
0,2,-11.971162,2.161283,2.187325,2.718785,-2.376041,-12.380814,1.684510,2.965321,0.283479,-4.926489,wt,1.095721,2.783979,3.49435,1.837332,ND,-0.980536,2.459135,3.734644,ND,-11.640326,0.129946
1,3,-5.486568,-3.240479,0.839046,-0.033629,2.640954,2.892552,-3.208603,2.110597,1.504923,-0.072862,-1.661449,wt,2.029029,1.119652,1.34098,1.310335,-0.974477,0.804706,0.006562,1.565156,1.435336,0.984125
2,4,-5.283314,-1.561144,0.540686,1.155679,2.696174,2.763591,0.444616,-0.301261,-1.449977,-1.447484,-11.272328,wt,-0.33404,-0.876281,-2.150697,-2.986365,-1.962691,-2.595102,-1.080793,-1.197572,0.558868,0.027498
3,5,-4.895570,-1.215611,1.09342,-0.744586,1.137211,0.905211,0.632540,-4.048257,-1.475024,0.001244,-5.697418,-0.383721,0.108542,-2.773253,-0.95423,-1.587203,-7.251361,wt,-1.662412,-1.150851,-1.97691,-0.258002
4,6,-12.731851,ND,-1.649598,-0.903615,1.673361,-0.372256,-2.218105,-1.997313,-0.796312,-1.151713,-2.937638,-0.568388,-1.265976,-2.645655,-1.489834,-1.887918,-0.867155,wt,-3.027533,-2.088556,-9.881243,-1.467015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,61,-3.934877,-12.957425,0.607752,0.005399,-14.009332,-10.637187,-4.836798,-5.993676,-5.694806,-2.652822,-1.7506,wt,0.041294,-9.897403,-7.976543,-10.666542,-13.39517,-3.680073,-11.860197,1.37174,-12.785212,-1.551672
60,62,-3.470346,-6.107599,0.094204,0.474593,0.449551,wt,0.574692,0.512504,0.105413,-0.949045,-2.046615,1.162255,0.539671,-1.543744,-14.334375,-0.119458,-0.739709,-0.574011,0.697578,1.664991,1.148038,0.219194
61,63,-11.368564,-13.897321,1.874195,-0.559317,-12.141996,-11.411682,1.425094,-3.409751,-3.180938,1.823171,-1.679518,0.999547,0.857151,-3.832084,-2.515373,0.288093,wt,-1.055604,0.762966,1.877127,-6.408608,-10.48212
62,64,-7.168358,-1.256885,-11.127599,-3.40978,-12.453882,-10.567995,0.055058,ND,-7.004315,1.497151,-12.130951,wt,0.208275,-12.411452,-10.778608,-12.682902,-3.125247,-12.558383,-12.966481,-11.523427,-12.383276,-10.564695


In [5]:
cols = ['*', 'delInFrame', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
data = pd.melt(data, id_vars='Residue', value_vars=cols)

data = data[data['value'] != 'ND']
data = data[data['value'] != 'wt']
data.rename(columns={'Residue': 'position', 'variable': 'mut_AA', 'value': 'enrichment_log2ratio'}, inplace=True)
data

,position,mut_AA,enrichment_log2ratio
0,2,*,-11.971162
1,3,*,-5.486568
2,4,*,-5.283314
3,5,*,-4.89557
4,6,*,-12.731851
...,...,...,...
1403,61,Y,-1.551672
1404,62,Y,0.219194
1405,63,Y,-10.48212
1406,64,Y,-10.564695


In [6]:
data['wt_AA'] = [gal4_wt[i-1] for i in data['position']]
data['mutation'] = data['wt_AA'].astype(str) + data['position'].astype(str) + data['mut_AA'].astype(str)
data

,position,mut_AA,enrichment_log2ratio,wt_AA,mutation
0,2,*,-11.971162,K,K2*
1,3,*,-5.486568,L,L3*
2,4,*,-5.283314,L,L4*
3,5,*,-4.89557,S,S5*
4,6,*,-12.731851,S,S6*
...,...,...,...,...,...
1403,61,Y,-1.551672,L,L61Y
1404,62,Y,0.219194,E,E62Y
1405,63,Y,-10.48212,R,R63Y
1406,64,Y,-10.564695,L,L64Y


In [7]:
import re

def split_string(input_string):
    #pattern = r'([A-Za-z]+)(\d+)([A-Za-z]+)'
    pattern = r'([A-Za-z]+)(\d+)([A-Za-z*]+)'
    matches = re.match(pattern, input_string)
    if matches:
        return [matches.group(1), int(matches.group(2)), matches.group(3)]
    else:
        # Return some default value, such as an empty list
        return []


def mutated_seqs(mutation, wt):
    seq = wt
    mut = split_string(mutation)
    wt_AA = mut[0]
    coor = int(mut[1]) - 1
    sub = mut[2]

    if '*' in mut[2] and seq[coor] == wt_AA:
        seq = seq[:coor]
    
    elif 'delInFrame' in mut[2] and seq[coor] == wt_AA:
        seq = seq[:coor] + seq[coor + 1:]
    
    else:
        if seq[coor] == wt_AA:
            seq = seq[:coor] + sub + seq[coor + 1:]
    
    return seq

# Testing the function
print(mutated_seqs('S6*', gal4_wt))
print(mutated_seqs('S5delInFrame', gal4_wt))
print(mutated_seqs('S5A', gal4_wt))
print(mutated_seqs('K27*', gal4_wt))


MKLLS
MKLLSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKRSPLTRAHLTEVESRLERLEQLFLLIFPREDLDMILKMDSLQDIKALLTGLFVQDNVNKDAVTDRLASVETDMPLTLRQHRISATSSSEESSNKGQRQLTVSIDSAAHHDNSTIPLDFMPRDALHGFDWSEEDDMSDGLPFLKTDPNNNGFFGDGSLLCILRSIGFKPENYTNSNVNRLPTMITDRYTLASRSTTSRLLQSYLNNFHPYCPIVHSPTLMMLYNNQIEIASKDQWQILFNCILAIGAWCIEGESTDIDVFYYQNAKSHLTSKVFESGSIILVTALHLLSRYTQWRQKTNTSYNFHSFSIRMAISLGLNRDLPSSFSDSSILEQRRRIWWSVYSWEIQLSLLYGRSIQLSQNTISFPSSVDDVQRTTTGPTIYHGIIETARLLQVFTKIYELDKTVTAEKSPICAKKCLMICNEIEEVSRQAPKFLQMDISTTALTNLLKEHPWLSFTRFELKWKQLSLIIYVLRDFFTNFTQKKSQLEQDQNDHQSYEVKRCSIMLSDAAQRTVMSVSSYMDNHNVTPYFAWNCSYYLFNAVLVPIKTLLSNSKSNAENNETAQLLQQINTVLMLLKKLATFKIQTCEKYIQVLEEVCAPFLLSQCAIPLPHISYNNSNGSAIKNIVGSATIAQYPTLPEENVNNISVKYVSPGSVGPSPVPLKSGASFSDLVKLLSNRPPSRNSPVTIPRSTPSHRSVTPFLGQQQQLQSLVPLTPSALFGGANFNQSGNIADSSLSFTFTNSSNGPNLITTQTNSQALSQPIASSNVHDNFMNNEITASKIDDGNNSKPLSPGWTDQTAYNAFGITTGMFNTTTMDDVYNYLFDDEDTPPNPKKE
MKLLASIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKRSPLTRAHLTEVESRLERLEQLFLLIFPREDLDMILKMDSLQDIKALLTGLFVQDNVNKDAVTDRLAS

In [8]:
data['sequences'] = data['mutation'].apply(mutated_seqs, args=(gal4_wt,))
data

,position,mut_AA,enrichment_log2ratio,wt_AA,mutation,sequences
0,2,*,-11.971162,K,K2*,M
1,3,*,-5.486568,L,L3*,MK
2,4,*,-5.283314,L,L4*,MKL
3,5,*,-4.89557,S,S5*,MKLL
4,6,*,-12.731851,S,S6*,MKLLS
...,...,...,...,...,...,...
1403,61,Y,-1.551672,L,L61Y,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...
1404,62,Y,0.219194,E,E62Y,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...
1405,63,Y,-10.48212,R,R63Y,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...
1406,64,Y,-10.564695,L,L64Y,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...


In [9]:
data['ID'] = 'GAL4_' + data['mutation'].astype(str)
data['ID'] = data['ID'].str.replace('*', '', regex=True)
data

,position,mut_AA,enrichment_log2ratio,wt_AA,mutation,sequences,ID
0,2,*,-11.971162,K,K2*,M,GAL4_K2
1,3,*,-5.486568,L,L3*,MK,GAL4_L3
2,4,*,-5.283314,L,L4*,MKL,GAL4_L4
3,5,*,-4.89557,S,S5*,MKLL,GAL4_S5
4,6,*,-12.731851,S,S6*,MKLLS,GAL4_S6
...,...,...,...,...,...,...,...
1403,61,Y,-1.551672,L,L61Y,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...,GAL4_L61Y
1404,62,Y,0.219194,E,E62Y,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...,GAL4_E62Y
1405,63,Y,-10.48212,R,R63Y,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...,GAL4_R63Y
1406,64,Y,-10.564695,L,L64Y,MKLLSSIEQACDICRLKKLKCSKEKPKCAKCLKNNWECRYSPKTKR...,GAL4_L64Y


In [10]:
# Create the fasta file
dict_seq = dict(zip(data['ID'], data['sequences']))

for k,v in dict_seq.items():
    with open('GAL4_SEL_C_64h/gal4_sel_c_64h_mutants_data.fasta', 'a') as f:
        f.write(f'>{k}\n{v}\n') 

In [11]:
cols = ['ID', 'mutation', 'wt_AA', 'position', 'mut_AA', 'enrichment_log2ratio',  'sequences']
data[cols].to_csv('GAL4_SEL_C_64h/gal4_sel_c_64h_mutants_metadata.csv', index=False)

# Blat Ecolx Ostermeier

In [2]:
wt_seq = str(SeqIO.read('/Users/luiz/Downloads/multi_layer_analysis/Data_process/BLAT_ECOLX_Ostermeier2014/rcsb_pdb_1ZG4.fasta', 'fasta').seq)
len(wt_seq)

286

In [3]:
#data = pd.read_excel('/Users/luiz/Downloads/multi_layer_analysis/Data_process/BLAT_ECOLX_Ostermeier2014/Data_S1-S4.xlsx', sheet_name=1, header=0)
cols = ['Ambler Position', 'WT AA', 'Mutant AA', 'Total Counts', 'Fitness', 'Estimated error in fitness']
data = pd.read_excel('/Users/luiz/Downloads/multi_layer_analysis/Data_process/BLAT_ECOLX_Ostermeier2014/Data_S1-S4.xlsx', sheet_name=1, usecols=cols)
data = data.drop(0)
data.rename(columns={'WT AA': 'wt_AA', 'Mutant AA': 'mut_AA', 'Total Counts': 'total_counts', 'Fitness': 'fitness', 'Estimated error in fitness': 'fitness_error'}, inplace=True)
data['position'] = data['Ambler Position'].astype(int) -2
#data['position'] = data['position'].astype(int)
data = data[~data['fitness'].isna()]
data

/Users/luiz/anaconda3/envs/ML/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Ambler Position,wt_AA,mut_AA,total_counts,fitness,fitness_error,position
1,3.0,M,A,51.0,0.002018,0.000294,1
2,3.0,M,C,33.0,0.003023,0.000522,1
3,3.0,M,D,12.0,0.002220,0.000567,1
4,3.0,M,E,31.0,0.002378,0.000421,1
5,3.0,M,F,10.0,0.005702,0.001562,1
...,...,...,...,...,...,...,...
5732,291.0,*,N,12.0,0.928524,0.237003,289
5735,291.0,*,R,21.0,0.613156,0.126042,289
5737,291.0,*,T,20.0,0.791548,0.165813,289
5738,291.0,*,V,25.0,0.851460,0.163613,289


In [4]:
# their data set has some positions wrong, so I'm fixing it here.

# first find the wrong positions
np.setxor1d(data['position'].unique(), np.array([i for i in range(1, 288)]))

array([237, 251, 288, 289])

In [5]:
data['position'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [6]:
def change_position(x):
    if x > 236 and x <= 250:
        x = x - 1
    elif x > 250:
        x = x - 2
    return x
change_position(242)

241

In [7]:
data['position'] = data['position'].apply(change_position)

np.setxor1d(np.array(data['position'].values), np.array([i for i in range(1, 288)]))

array([], dtype=int64)

In [8]:
data['mutation'] = data['wt_AA'].astype(str) + data['position'].astype(str) + data['mut_AA'].astype(str)
data

,Ambler Position,wt_AA,mut_AA,total_counts,fitness,fitness_error,position,mutation
1,3.0,M,A,51.0,0.002018,0.000294,1,M1A
2,3.0,M,C,33.0,0.003023,0.000522,1,M1C
3,3.0,M,D,12.0,0.002220,0.000567,1,M1D
4,3.0,M,E,31.0,0.002378,0.000421,1,M1E
5,3.0,M,F,10.0,0.005702,0.001562,1,M1F
...,...,...,...,...,...,...,...,...
5732,291.0,*,N,12.0,0.928524,0.237003,287,*287N
5735,291.0,*,R,21.0,0.613156,0.126042,287,*287R
5737,291.0,*,T,20.0,0.791548,0.165813,287,*287T
5738,291.0,*,V,25.0,0.851460,0.163613,287,*287V


In [9]:
import re

def split_string(input_string):
    pattern = r'([A-Za-z*]+)(\d+)([A-Za-z*]+)'
    matches = re.match(pattern, input_string)
    if matches:
        return [matches.group(1), int(matches.group(2)), matches.group(3)]
    else:
        # Return some default value, such as an empty list
        return []


def mutated_seqs(mutation, wt):
    seq = wt
    mut = split_string(mutation)
    wt_AA = mut[0]
    coor = int(mut[1]) -1
    sub = mut[2]

    if '*' in mut[2] and seq[coor] == wt_AA:
        seq = seq[:coor]
    
    elif '*' in mut[0]:
        seq = seq + sub
    
    else:
        if seq[coor] == wt_AA:
            seq = seq[:coor] + sub + seq[coor + 1:]
    
    return seq

# Testing the function
print(wt_seq)
print(mutated_seqs('H5L', wt_seq))
print(mutated_seqs('*287Y', wt_seq))

MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRIDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPVAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW
MSIQLFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRIDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPVAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW
MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRIDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPVAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHWY


In [10]:
data['sequence'] = data['mutation'].apply(mutated_seqs, args=(wt_seq,))
data

,Ambler Position,wt_AA,mut_AA,total_counts,fitness,fitness_error,position,mutation,sequence
1,3.0,M,A,51.0,0.002018,0.000294,1,M1A,ASIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
2,3.0,M,C,33.0,0.003023,0.000522,1,M1C,CSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
3,3.0,M,D,12.0,0.002220,0.000567,1,M1D,DSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
4,3.0,M,E,31.0,0.002378,0.000421,1,M1E,ESIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
5,3.0,M,F,10.0,0.005702,0.001562,1,M1F,FSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
...,...,...,...,...,...,...,...,...,...
5732,291.0,*,N,12.0,0.928524,0.237003,287,*287N,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
5735,291.0,*,R,21.0,0.613156,0.126042,287,*287R,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
5737,291.0,*,T,20.0,0.791548,0.165813,287,*287T,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
5738,291.0,*,V,25.0,0.851460,0.163613,287,*287V,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...


In [11]:
data['ID'] = 'BLac_' + data['mutation'].astype(str)
cols = ['ID', 'Ambler Position', 'wt_AA', 'position', 'mut_AA', 'mutation', 'total_counts', 'fitness', 'fitness_error',   'sequence']
data = data[cols]
data

,ID,Ambler Position,wt_AA,position,mut_AA,mutation,total_counts,fitness,fitness_error,sequence
1,BLac_M1A,3.0,M,1,A,M1A,51.0,0.002018,0.000294,ASIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
2,BLac_M1C,3.0,M,1,C,M1C,33.0,0.003023,0.000522,CSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
3,BLac_M1D,3.0,M,1,D,M1D,12.0,0.002220,0.000567,DSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
4,BLac_M1E,3.0,M,1,E,M1E,31.0,0.002378,0.000421,ESIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
5,BLac_M1F,3.0,M,1,F,M1F,10.0,0.005702,0.001562,FSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
...,...,...,...,...,...,...,...,...,...,...
5732,BLac_*287N,291.0,*,287,N,*287N,12.0,0.928524,0.237003,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
5735,BLac_*287R,291.0,*,287,R,*287R,21.0,0.613156,0.126042,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
5737,BLac_*287T,291.0,*,287,T,*287T,20.0,0.791548,0.165813,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...
5738,BLac_*287V,291.0,*,287,V,*287V,25.0,0.851460,0.163613,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...


In [12]:
# # Create the fasta file
# dict_seq = dict(zip(data['ID'], data['sequence']))

# for k,v in dict_seq.items():
#     with open('/Users/luiz/Downloads/multi_layer_analysis/Data_process/BLAT_ECOLX_Ostermeier2014/BLAT_ECOLX_Ostermeier2014_data.fasta', 'a') as f:
#         f.write(f'>{k}\n{v}\n') 

In [13]:
#data.to_csv('/Users/luiz/Downloads/multi_layer_analysis/Data_process/BLAT_ECOLX_Ostermeier2014/BLAT_ECOLX_Ostermeier2014_metadata.csv', index=False)

# Blac - Ranganathan 2015

In [20]:
wt_seq = str(SeqIO.read('/Users/luiz/Downloads/multi_layer_analysis/Data_process/BLAT_ECOLX_Ostermeier2014/rcsb_pdb_1ZG4.fasta', 'fasta').seq)
wt_seq

'MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRIDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPVAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW'

In [112]:
data = pd.read_excel('/Users/luiz/Downloads/multi_layer_analysis/Data_process/BLAT_ECOLX_Ranganathan2015/mmc1.xlsx', sheet_name=1)
data

,mutation,2500_T1,2500_T2
0,H26A,-0.003758,-0.015810
1,H26C,-0.319357,-0.517163
2,H26D,-0.155938,-0.207275
3,H26E,-0.200042,-0.358006
4,H26F,-0.493982,-1.142992
...,...,...,...
4992,W290R,-2.381525,-3.033937
4993,W290S,-2.463131,-3.472294
4994,W290T,-2.380080,-3.232764
4995,W290V,-2.535303,-3.092371


In [127]:
data['mean_fitness'] = data[['2500_T1','2500_T2']].mean(axis=1)
data

,mutation,2500_T1,2500_T2,mean_fitness
0,H26A,-0.003758,-0.015810,-0.009784
1,H26C,-0.319357,-0.517163,-0.418260
2,H26D,-0.155938,-0.207275,-0.181607
3,H26E,-0.200042,-0.358006,-0.279024
4,H26F,-0.493982,-1.142992,-0.818487
...,...,...,...,...
4992,W290R,-2.381525,-3.033937,-2.707731
4993,W290S,-2.463131,-3.472294,-2.967712
4994,W290T,-2.380080,-3.232764,-2.806422
4995,W290V,-2.535303,-3.092371,-2.813837


In [136]:
len(wt_seq)

286

In [137]:
290-3

287

In [ ]:
import re

def split_string(input_string):
    pattern = r'([A-Za-z*]+)(\d+)([A-Za-z*]+)'
    matches = re.match(pattern, input_string)
    if matches:
        return [matches.group(1), int(matches.group(2)), matches.group(3)]
    else:
        # Return some default value, such as an empty list
        return []


def mutated_seqs(mutation, wt):
    seq = wt
    mut = split_string(mutation)
    wt_AA = mut[0]
    coor = int(mut[1]) -1
    sub = mut[2]

    if '*' in mut[2] and seq[coor] == wt_AA:
        seq = seq[:coor]
    
    elif '*' in mut[0]:
        seq = seq + sub
    
    else:
        if seq[coor] == wt_AA:
            seq = seq[:coor] + sub + seq[coor + 1:]
    
    return seq

# Testing the function
print(wt_seq)
print(mutated_seqs('H26L', wt_seq))
print(mutated_seqs('*287Y', wt_seq))

# CALM1 screen scores

In [143]:
calm1 = str(SeqIO.read('/Users/luiz/Downloads/multi_layer_analysis/Data_process/CALM1_HUMAN/calm1_human.fasta', 'fasta').seq)
calm1

'MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTIDFPEFLTMMARKMKDTDSEEEIREAFRVFDKDGNGYISAAELRHVMTNLGEKLTDEEVDEMIREADIDGDGQVNYEEFVQMMTAK'

In [141]:
data = pd.read_csv('CALM1_HUMAN/CALM1_scores.csv').dropna(subset=['screen.score'])
data

,mut,screen.score,screen.sd,df,predicted.score,joint.score,joint.sd,joint.se
1,A2A,1.000000,0.000100,4.0,1.008986,1.000000,0.000100,0.000100
2,D3A,0.620463,0.043916,4.0,0.673603,0.623796,0.093411,0.046706
4,L5A,0.505060,0.045762,4.0,0.577514,0.509784,0.096411,0.048205
5,T6A,1.011058,0.010346,4.0,1.010209,1.011045,0.042145,0.021073
6,E7A,0.674627,0.025012,4.0,0.738476,0.676971,0.068551,0.034275
...,...,...,...,...,...,...,...,...
2968,N138P,1.274694,0.021322,4.0,1.089746,1.268874,0.069789,0.034894
2969,Y139P,0.609328,0.089303,4.0,0.548164,0.602001,0.142508,0.071254
2974,Q144P,0.909449,0.024116,4.0,0.819118,0.906247,0.068233,0.034116
2977,T147P,1.006368,0.014258,4.0,0.930589,1.004757,0.051065,0.025533


In [164]:
import re

def split_string(input_string):
    pattern = r'([A-Za-z*]+)(\d+)([A-Za-z*]+)'
    matches = re.match(pattern, input_string)
    if matches:
        return [matches.group(1), int(matches.group(2)), matches.group(3)]
    else:
        # Return some default value, such as an empty list
        return []


def mutated_seqs(mutation, wt):
    seq = wt
    mut = split_string(mutation)
    wt_AA = mut[0]
    coor = int(mut[1]) -1
    sub = mut[2]

    if '*' in mut[2] and seq[coor] == wt_AA:
        seq = seq[:coor]
    
    elif '*' in mut[0]:
        seq = seq + sub
    
    else:
        if seq[coor] == wt_AA:
            seq = seq[:coor] + sub + seq[coor + 1:]
    
    return seq


print(caml1)
print(mutated_seqs('D3A', calm1))  
print(mutated_seqs('E7A', calm1))  

MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTIDFPEFLTMMARKMKDTDSEEEIREAFRVFDKDGNGYISAAELRHVMTNLGEKLTDEEVDEMIREADIDGDGQVNYEEFVQMMTAK
MAAQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTIDFPEFLTMMARKMKDTDSEEEIREAFRVFDKDGNGYISAAELRHVMTNLGEKLTDEEVDEMIREADIDGDGQVNYEEFVQMMTAK
MADQLTAEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTIDFPEFLTMMARKMKDTDSEEEIREAFRVFDKDGNGYISAAELRHVMTNLGEKLTDEEVDEMIREADIDGDGQVNYEEFVQMMTAK


In [147]:
data['sequence'] = data['mut'].apply(mutated_seqs, args=(calm1,))
data

,mut,screen.score,screen.sd,df,predicted.score,joint.score,joint.sd,joint.se,sequence
1,A2A,1.000000,0.000100,4.0,1.008986,1.000000,0.000100,0.000100,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
2,D3A,0.620463,0.043916,4.0,0.673603,0.623796,0.093411,0.046706,MAAQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
4,L5A,0.505060,0.045762,4.0,0.577514,0.509784,0.096411,0.048205,MADQATEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
5,T6A,1.011058,0.010346,4.0,1.010209,1.011045,0.042145,0.021073,MADQLAEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
6,E7A,0.674627,0.025012,4.0,0.738476,0.676971,0.068551,0.034275,MADQLTAEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
...,...,...,...,...,...,...,...,...,...
2968,N138P,1.274694,0.021322,4.0,1.089746,1.268874,0.069789,0.034894,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
2969,Y139P,0.609328,0.089303,4.0,0.548164,0.602001,0.142508,0.071254,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
2974,Q144P,0.909449,0.024116,4.0,0.819118,0.906247,0.068233,0.034116,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
2977,T147P,1.006368,0.014258,4.0,0.930589,1.004757,0.051065,0.025533,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...


In [167]:
data['ID'] = 'CALM1_' + data['mut'].astype(str)
cols = ['ID', 'mut', 'screen.score', 'sequence']
data = data[cols]
data

,ID,mut,screen.score,sequence
1,CALM1_A2A,A2A,1.000000,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
2,CALM1_D3A,D3A,0.620463,MAAQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
4,CALM1_L5A,L5A,0.505060,MADQATEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
5,CALM1_T6A,T6A,1.011058,MADQLAEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
6,CALM1_E7A,E7A,0.674627,MADQLTAEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
...,...,...,...,...
2968,CALM1_N138P,N138P,1.274694,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
2969,CALM1_Y139P,Y139P,0.609328,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
2974,CALM1_Q144P,Q144P,0.909449,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
2977,CALM1_T147P,T147P,1.006368,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...


In [168]:
# Create the fasta file
dict_seq = dict(zip(data['ID'], data['sequence']))

for k,v in dict_seq.items():
    with open('/Users/luiz/Downloads/multi_layer_analysis/Data_process/CALM1_HUMAN/calm1_human_data.fasta', 'a') as f:
        f.write(f'>{k}\n{v}\n') 

In [ ]:
data.to_csv('/Users/luiz/Downloads/multi_layer_analysis/Data_process/CALM1_HUMAN/calm1_human_metadata.csv')